# Radar Frame Labeling Tool
Samples events from the TorNet catalog and lets you label each frame as:
- `tornado_hook`
- `tornado_no_hook`
- `non_tornadic`

Results are saved to a CSV after every label.

In [1]:
import os
import random
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

## Configuration

In [ ]:
# s3_client = boto3.client('s3')
# bucket_name = 'ml-cloud-project-data'

In [6]:
# ── Change these to match your setup ──────────────────────────────────────────
DATA_DIR    = "s3://ml-cloud-project-data/"          # root dir containing tornet_2013/, tornet_2014/, etc.
CATALOG_CSV = "s3://ml-cloud-project-data/tornet_2013/catalog.csv"
SPLIT            = "train"               # 'train' or 'test'
N_PER_CATEGORY   = 50                    # unique events (by event_id) per category
FRAMES_PER_EVENT = None                  # frames to subsample per event (None = all)
OUTPUT_CSV       = "labels.csv"
SWEEP_IDX        = 0                     # which sweep to display (0 = lowest elevation)
SEED             = 42
# ──────────────────────────────────────────────────────────────────────────────

VARIABLES_TO_PLOT = ['DBZ', 'VEL']

COLORMAP_MAP = {
    'DBZ': ('gist_ncar', -10, 75),
    'VEL': ('bwr',       -30, 30),
}

LABEL_OPTIONS = ['tornado_hook', 'tornado_no_hook', 'non_tornadic']

## Event Sampling from Catalog

In [7]:
def sample_events_from_catalog(catalog_csv, data_dir, split, n_per_category, seed=42):
    """
    Read the TorNet catalog and sample n_per_category unique tornado events (TOR)
    and n_per_category non-tornadic events (NUL).

    The catalog 'filename' column contains relative paths like:
        train/2013/TOR_130901_222639_KBOX_469573_M0.nc
    These are resolved to absolute paths using DATA_DIR and the year-based folder structure.
    """
    random.seed(seed)
    catalog = pd.read_csv(catalog_csv)
    catalog = catalog[catalog['type'] == split].copy()

    def sample_category(cat_label, n):
        rows = catalog[catalog['category'] == cat_label]
        unique_ids = rows['event_id'].unique().tolist()
        sampled_ids = random.sample(unique_ids, min(n, len(unique_ids)))
        # One representative file per event_id
        files = (
            rows[rows['event_id'].isin(sampled_ids)]
            .groupby('event_id')['filename']
            .first()
            .tolist()
        )
        print(f"  {cat_label}: {len(files)} events sampled from {len(unique_ids)} available")
        return files

    print(f"Sampling from split='{split}' ...")
    tor_files = sample_category('TOR', n_per_category)
    nul_files = sample_category('NUL', n_per_category)

    def resolve_path(rel_filename):
        # rel_filename: 'train/2013/TOR_....nc'
        parts = Path(rel_filename).parts   # ('train', '2013', 'filename.nc')
        year  = parts[1] if len(parts) >= 3 else ''
        return os.path.join(data_dir, f"tornet_{year}", rel_filename)

    all_files = [resolve_path(f) for f in tor_files + nul_files]
    random.shuffle(all_files)
    return all_files


def load_event_frames(filepath, frames_per_event=None, seed=42):
    """Open a .nc file and return (dataset, list_of_frame_indices)."""
    ds = xr.open_dataset(filepath, engine='h5netcdf')
    n_frames = ds.dims.get('time', 1)
    indices  = list(range(n_frames))
    if frames_per_event and len(indices) > frames_per_event:
        random.seed(seed)
        indices = sorted(random.sample(indices, frames_per_event))
    return ds, indices


all_event_files = sample_events_from_catalog(
    CATALOG_CSV, DATA_DIR, SPLIT, N_PER_CATEGORY, SEED
)
print(f"\nTotal events queued : {len(all_event_files)}")
print(f"Example path        : {all_event_files[0] if all_event_files else 'n/a'}")

Sampling from split='train' ...
  TOR: 50 events sampled from 6655 available
  NUL: 50 events sampled from 13620 available

Total events queued : 100
Example path        : s3://ml-cloud-project-data/tornet_2014/train/2014/TOR_140404_020014_KPAH_504204_P0.nc


## Labeling UI

In [8]:
def plot_frame(ds, frame_idx, event_name, frame_num, total_frames):
    """Render DBZ and VEL for one frame. Data shape: (time, azimuth, range, sweep)."""
    vars_present = [v for v in VARIABLES_TO_PLOT if v in ds]
    fig, axes = plt.subplots(1, len(vars_present), figsize=(10, 4.5), facecolor='#0d0d0d')
    if len(vars_present) == 1:
        axes = [axes]

    fig.suptitle(f"{event_name}  |  Frame {frame_num}/{total_frames}",
                 color='white', fontsize=12, fontweight='bold')

    for ax, var in zip(axes, vars_present):
        ax.set_facecolor('#111')
        data = ds[var].values   # (time, azimuth, range, sweep)

        if data.ndim == 4:
            frame_data = data[frame_idx, :, :, SWEEP_IDX]
        elif data.ndim == 3:
            frame_data = data[frame_idx, :, :]
        else:
            frame_data = data

        cmap, vmin, vmax = COLORMAP_MAP.get(var, ('viridis', None, None))
        im = ax.imshow(frame_data, cmap=cmap, vmin=vmin, vmax=vmax,
                       origin='lower', aspect='auto', interpolation='nearest')
        cb = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cb.ax.tick_params(colors='white')
        ax.set_title(var, color='white', fontsize=11)
        ax.set_xlabel('Range', color='white', fontsize=8)
        ax.set_ylabel('Azimuth', color='white', fontsize=8)
        ax.tick_params(colors='white')
        for spine in ax.spines.values():
            spine.set_edgecolor('#333')

    plt.tight_layout()
    return fig

In [9]:
class NotebookLabeler:
    """Widget-based frame labeling tool. Call .start() to begin."""

    def __init__(self, event_files, output_csv=OUTPUT_CSV, frames_per_event=FRAMES_PER_EVENT):
        self.event_files      = event_files
        self.output_csv       = output_csv
        self.frames_per_event = frames_per_event
        self.results          = []
        self._queue           = []
        self._pos             = 0
        self._ds_cache        = {}

        # ── Widgets ──────────────────────────────────────────────
        self.btn_hook   = widgets.Button(description='🌀 Tornado w/ Hook',
                                         layout=widgets.Layout(width='200px', height='44px'),
                                         style=dict(button_color='#1a3a6b', font_weight='bold'))
        self.btn_nohook = widgets.Button(description='⚡ Tornado No Hook',
                                         layout=widgets.Layout(width='200px', height='44px'),
                                         style=dict(button_color='#1a5a6b', font_weight='bold'))
        self.btn_non    = widgets.Button(description='☁️ Non-Tornadic',
                                         layout=widgets.Layout(width='200px', height='44px'),
                                         style=dict(button_color='#2d4a1e', font_weight='bold'))
        self.btn_skip   = widgets.Button(description='⏭ Skip Event',
                                         layout=widgets.Layout(width='130px', height='44px'),
                                         style=dict(button_color='#4a3800', font_weight='bold'))
        self.btn_undo   = widgets.Button(description='↩ Undo',
                                         layout=widgets.Layout(width='100px', height='44px'),
                                         style=dict(button_color='#3a1e3a', font_weight='bold'))

        self.progress_bar   = widgets.IntProgress(min=0, max=1, value=0,
                                                   layout=widgets.Layout(width='90%'),
                                                   style={'bar_color': '#2a72c8'})
        self.progress_label = widgets.Label(value='0/0 frames')
        self.status_label   = widgets.Label()
        self.plot_output    = widgets.Output()

        self.btn_hook.on_click(lambda b:   self._label('tornado_hook'))
        self.btn_nohook.on_click(lambda b: self._label('tornado_no_hook'))
        self.btn_non.on_click(lambda b:    self._label('non_tornadic'))
        self.btn_skip.on_click(lambda b:   self._skip_event())
        self.btn_undo.on_click(lambda b:   self._undo())

        self._ui = widgets.VBox([
            widgets.HBox([self.progress_bar, self.progress_label]),
            self.plot_output,
            widgets.HBox(
                [self.btn_hook, self.btn_nohook, self.btn_non, self.btn_skip, self.btn_undo],
                layout=widgets.Layout(justify_content='center', gap='8px', margin='8px 0')
            ),
            self.status_label,
        ])

    def start(self):
        self._build_queue()
        if not self._queue:
            print("Queue is empty — check that DATA_DIR and CATALOG_CSV are correct.")
            return
        self.progress_bar.max = len(self._queue)
        display(self._ui)
        self._show_current()

    @property
    def labels_df(self):
        return pd.DataFrame(self.results)

    def _build_queue(self):
        for fp in self.event_files:
            try:
                ds, indices = load_event_frames(fp, self.frames_per_event, SEED)
                self._ds_cache[fp] = (ds, indices)
                for fi in indices:
                    self._queue.append((fp, fi))
            except Exception as e:
                print(f"Could not load {Path(fp).name}: {e}")
        print(f"Total frames to label: {len(self._queue)}")

    def _show_current(self):
        if self._pos >= len(self._queue):
            self._finish()
            return
        fp, frame_idx  = self._queue[self._pos]
        ds, indices    = self._ds_cache[fp]
        event_name     = Path(fp).stem
        frame_num      = indices.index(frame_idx) + 1
        total_frames   = len(indices)
        self.progress_bar.value   = self._pos
        self.progress_label.value = f"  {self._pos}/{len(self._queue)} frames"
        with self.plot_output:
            clear_output(wait=True)
            fig = plot_frame(ds, frame_idx, event_name, frame_num, total_frames)
            plt.show()

    def _label(self, label):
        if self._pos >= len(self._queue):
            return
        fp, frame_idx = self._queue[self._pos]
        self.results.append({
            'filepath':  fp,
            'event_id':  Path(fp).stem,
            'frame_idx': frame_idx,
            'label':     label,
        })
        self.status_label.value = f"✓ Labeled as: {label}"
        self._pos += 1
        self._auto_save()
        self._show_current()

    def _skip_event(self):
        if self._pos >= len(self._queue):
            return
        current_event = self._queue[self._pos][0]
        while self._pos < len(self._queue) and self._queue[self._pos][0] == current_event:
            self._pos += 1
        self.status_label.value = f"⏭ Skipped: {Path(current_event).stem}"
        self._show_current()

    def _undo(self):
        if self.results and self._pos > 0:
            self.results.pop()
            self._pos -= 1
            self.status_label.value = "↩ Undid last label"
            self._show_current()

    def _auto_save(self):
        if self.results:
            pd.DataFrame(self.results).to_csv(self.output_csv, index=False)

    def _finish(self):
        self._auto_save()
        with self.plot_output:
            clear_output(wait=True)
            print("✅ All frames labeled!")
        self.progress_bar.value   = len(self._queue)
        self.progress_label.value = f"  {len(self._queue)}/{len(self._queue)} frames"
        self.status_label.value   = f"💾 Saved to {self.output_csv}"
        df = pd.DataFrame(self.results)
        print("\n── Label distribution ──")
        print(df['label'].value_counts().to_string())
        print(f"\nTotal labeled frames: {len(df)}")

## Run the Labeler
Click the buttons beneath each frame to assign a label. Progress autosaves after every label.

In [10]:
labeler = NotebookLabeler(
    event_files=all_event_files,
    output_csv=OUTPUT_CSV,
    frames_per_event=FRAMES_PER_EVENT,
)
labeler.start()

/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('time', 1)
<frozen _collections_abc>:774: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('tim

Could not load NUL_170309_204836_KEAX_686492s_H2.nc: ml-cloud-project-data/tornet_2017/train/2017/NUL_170309_204836_KEAX_686492s_H2.nc


/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('time', 1)
<frozen _collections_abc>:774: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('tim

Could not load TOR_170121_163418_KFFC_673683_I2.nc: ml-cloud-project-data/tornet_2017/train/2017/TOR_170121_163418_KFFC_673683_I2.nc


/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('time', 1)
<frozen _collections_abc>:774: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('tim

...

/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('time', 1)
<frozen _collections_abc>:774: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('tim

Could not load NUL_171022_100350_KLIX_1079872n_V4.nc: ml-cloud-project-data/tornet_2017/train/2017/NUL_171022_100350_KLIX_1079872n_V4.nc


/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('time', 1)
<frozen _collections_abc>:774: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('tim

Total frames to label: 384


/var/folders/1k/z2vtp8ks64j86zpxtb8f1bkh0000gn/T/ipykernel_49071/2549284638.py:46: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_frames = ds.dims.get('time', 1)
<frozen _collections_abc>:774: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.


## Review Labels

In [11]:
df = labeler.labels_df
print(df['label'].value_counts())
df.head()

label
non_tornadic       5
tornado_no_hook    3
Name: count, dtype: int64


,filepath,event_id,frame_idx,label
0,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,0,non_tornadic
1,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,1,tornado_no_hook
2,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,2,tornado_no_hook
3,s3://ml-cloud-project-data/tornet_2014/train/2...,TOR_140404_020014_KPAH_504204_P0,3,tornado_no_hook
4,s3://ml-cloud-project-data/tornet_2014/train/2...,NUL_140818_011753_KABR_540562s_P9,0,non_tornadic
